# Localiza desafio 1

O gerenciamento do relacionamento com os clientes tem como um dos seus objetivos
centrais aumentar o número de clientes ativos da empresa - e a retenção desses clientes é crítica
para seu sucesso. Sendo assim, um time foi mobilizado para garantir o desenvolvimento da
estratégia de retenção de clientes e, sabendo do seu potencial analítico, você foi convocado para
esse desafio.

Em busca de uma maior compreensão das particularidades do negócio e de melhor
entendimento do cenário do churn, vocês optaram por realizar uma análise exploratória dos
dados. Em um primeiro momento algumas hipóteses foram levantadas visando direcionar esse
desenvolvimento inicial e, considerando a base de dados disponibilizada e um período de churn
de 1 ano, foi requisitado a você a validação da seguinte hipótese:

“O churn de novos clientes é maior do que o churn de clientes ativos”

## Desenvolvimento

A hipótese será contestada diretamente pelo python notebook, observando-se algumas premissas e comparando os dois subgrupos dentro dos clientes.

 > o primeiro passo será carregar os dados em um dataframe (pandas) e explorar a base disponível

### Carregando dados

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd

with open('/gdrive/My Drive/Localiza/base_teste.csv', 'r') as csv:
  df = pd.read_csv(csv)

In [ ]:
df.head(5)

,Unnamed: 0,cidade_retirada,cidade_devolucao,cd_contrato,transporte_locado,data_abertura_contrato,qtd_diarias,data_fechamento_contrato,qtd_hora_extra,cd_cliente
0,0,Hogsmeade,Hogsmeade,HGM048798,Portal,2015-01-26 05:58:37,1,2015-01-27 05:58:37,0,1
1,1,Springfield,Springfield,SPF030834,Ônibus Escolar,2015-02-05 15:10:19,1,2015-02-06 15:10:19,0,1
2,2,Porto Real,Porto Real,PRL017782,Barco,2015-05-04 20:49:56,1,2015-05-05 20:49:56,0,1
3,3,Basin City,Basin City,SIN042621,1960 Corvette,2015-05-07 07:34:04,1,2015-05-08 07:34:04,0,1
4,4,Godric's Hollow,Godric's Hollow,GDH044954,Testralio,2015-08-01 11:14:48,2,2015-08-03 12:14:48,1,1


Temos em mão somente a base de locação de veículos, com detalhes da locação inclusive do id_cliente que locou o veículo. É o bastante para testarmos a hipótese, mas algumas premissas devem ser definidas:
  - **O que caracteriza um novo cliente e um cliente antigo?** 
  - **A partir de quanto tempo sem locar um transporte o cliente é considerado churn?**

### Premissas

  - **O que caracteriza um novo cliente e um cliente antigo?** A maneira mais fácil e óbvia de separar esses grupos é considerar um cliente como novo na primeira locação e antigo nas subsequentes. 
  - **A partir de quanto tempo sem locar um transporte o cliente é considerado churn?** Para isso é interessante explorar primeiro a recorrência de locações média de um cliente e se esse número não varia muito de cliente para cliente 

In [ ]:
metricas_cliente = df.groupby('cd_cliente').agg(primeiro_contrato=('data_abertura_contrato', min), ultimo_contrato=('data_abertura_contrato', max), qtd_contratos=('cd_contrato', 'size')).reset_index()

In [ ]:
metricas_cliente.head(5)

,cd_cliente,primeiro_contrato,ultimo_contrato,qtd_contratos
0,1,2020-10-20 21:26:18,2015-01-26 05:58:37,36
1,2,2020-12-12 05:39:35,2015-01-31 02:29:17,43
2,3,2020-10-31 18:15:45,2015-02-12 09:27:50,37
3,4,2020-12-13 17:31:26,2015-03-06 15:37:42,36
4,5,2020-12-11 08:37:12,2015-02-24 09:00:59,49


In [ ]:
##O Python lê as datas atualmente como strings, é necessário chamar a biblioteca datetime e usar uma conversão
import datetime

dateConversor = lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')

metricas_cliente['primeiro_contrato'] = metricas_cliente['primeiro_contrato'].apply(dateConversor)
metricas_cliente['ultimo_contrato'] = metricas_cliente['ultimo_contrato'].apply(dateConversor)

metricas_cliente['tempo_entre_contratos'] = (metricas_cliente['ultimo_contrato']-metricas_cliente['primeiro_contrato'])/metricas_cliente['qtd_contratos'] 

In [ ]:
# Usarei o leitor de expressões regulares para extrair somente o dia do tempo entre contratos
import re

dayExtractor = lambda x: int(re.search(r'^([0-9]*)',str(x)).group(1))

metricas_cliente['dias_entre_contratos'] = metricas_cliente['tempo_entre_contratos'].apply(dayExtractor)

In [ ]:
metricas_cliente.head(5)

,cd_cliente,primeiro_contrato,ultimo_contrato,qtd_contratos,tempo_entre_contratos,dias_entre_contratos
0,1,2015-01-26 05:58:37,2020-10-20 21:26:18,36,58 days 04:25:46.138888888,58
1,2,2015-01-31 02:29:17,2020-12-12 05:39:35,43,49 days 19:36:31.116279069,49
2,3,2015-02-12 09:27:50,2020-10-31 18:15:45,37,56 days 10:36:58.243243243,56
3,4,2015-03-06 15:37:42,2020-12-13 17:31:26,36,58 days 14:03:09.555555555,58
4,5,2015-02-24 09:00:59,2020-12-11 08:37:12,49,43 days 04:53:23.530612244,43


Uma vez que tenho a recorrência de cada cliente na *metricas_cliente* farei novas agregações para entender recorrência média dentre outras métricas estatísticas, e verificar seu comportamento na base.

In [ ]:
# chamando bilbioteca numpy par tirar os percentis
import numpy as np

agg = metricas_cliente[metricas_cliente.qtd_contratos > 1]['dias_entre_contratos']

print('mean: '+str(agg.mean()))
print('std: '+str(agg.std()))
print('min: '+str(agg.min()))
print('quartile 1: '+str(np.percentile(agg,25)))
print('median: '+str(np.percentile(agg,50)))
print('quartile 3: '+str(np.percentile(agg,75)))
print('max: '+str(agg.max()))

mean: 212.35822773948112
std: 165.84038961335872
min: 1
quartile 1: 89.0
median: 164.0
quartile 3: 283.0
max: 1078


Notamos que em média os clientes demoram 212 dias para realizar uma nova locação, porém esse número varia muito na base, sendo inseguro afirmar um churn a partir de 212 dias sem locação. Também notamos pela mediana e percentis que a grande maioria dos clientes tem intervalos de locação menores e alguns poucos puxam a média para cima.

Já que a hipótese é atemporal (não necessitamos dar peso maior para churns mais recente ou desconsiderar churns antigos), para definir o churn de forma segura usarei o percentil 80


In [ ]:
daysToChurn = np.percentile(agg,80)

print('percentile 80: '+str(daysToChurn))

percentile 80: 321.0


Ou seja, considerando **churn como cliente sem novo contrato a 321 dias** posso afirmar com 80% de certeza que se trata de um churn, pois **80% dos clientes já teriam feito nova locação nesse período**

### Cálculo de churn em cada grupo

Uma vez que estão definidos os grupos e o que é churn fica fácil separá-los. Começarei a partir do dataframe *metricas_cliente* trabalhado na última sessão, também usarei o *daysToChurn*, número de dias sem nova locação para considerar um churn, definido na última sessão.

In [ ]:
novoOuAntigo = lambda x: True if x == 1 else False

metricas_cliente['novo?'] = metricas_cliente['qtd_contratos'].apply(novoOuAntigo)

metricas_cliente.groupby('novo?').size()

novo?
False    34961
True      9881
dtype: int64

Como para aplicar o churn de cliente inativo a 321 dias, tenho que saber qual é o dia de "hoje", ou seja, até quando o CSV que estou usando tem dados e usá-lo como proxy do dia em que esses dados foram consultados. 

In [ ]:
metricas_cliente['ultimo_contrato'].max()

Timestamp('2020-12-31 23:54:45')

In [ ]:
churnTreshold = metricas_cliente['ultimo_contrato'].max() - datetime.timedelta(days=daysToChurn)

# Churn se a última locação foi antes da data corte definida acima
churnCalculator = lambda x: True if x < churnTreshold else False

metricas_cliente['churn?'] = metricas_cliente['ultimo_contrato'].apply(churnCalculator)

metricas_cliente.groupby('churn?').size()

churn?
False    25718
True     19124
dtype: int64

Por fim cruzarei ambas as definições de churn e novo / recorrente

In [ ]:
churn_por_grupo = metricas_cliente[['cd_cliente','novo?','churn?']].groupby(['novo?','churn?']).count()

churn_por_grupo_perc = metricas_cliente[['cd_cliente','novo?','churn?']].groupby(['novo?','churn?']).count()/metricas_cliente[['cd_cliente','novo?']].groupby('novo?').count()

churn_por_grupo = churn_por_grupo.rename(columns={'cd_cliente': 'qtd_clientes'})
churn_por_grupo_perc = churn_por_grupo_perc.rename(columns={'cd_cliente': 'taxa'})

churn_por_grupo['taxa'] = 0
churn_por_grupo_perc['qtd_clientes'] = 0

churn_por_grupo + churn_por_grupo_perc

qtd_clientes      taxa
novo? churn?                        
False False          24308  0.695289
      True           10653  0.304711
True  False           1410  0.142698
      True            8471  0.857302

## Conclusão



In [ ]:
churn_por_grupo + churn_por_grupo_perc

qtd_clientes      taxa
novo? churn?                        
False False          24308  0.695289
      True           10653  0.304711
True  False           1410  0.142698
      True            8471  0.857302

Podemos **validar a hipótese** levantada ao comparar que a taxa de churn dentro dos novos clientes (85%) é muito maior que a de clientes recorrentes (30%).

Dado que estamos trabalhando com populações inteiras e que a taxa encontrada é muito diferente, não há necessidade de realizarmos testes estatísticos sofisticados para comprovar essa hipótese.